# Prepare Data From CSV Recording

## Load File

Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.

The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.

The columns are reorded in the order of head/r_controller/l_controller/waist/r_foot/l_foot.

The new trimmed file is written to a directory (/test_data or /train_data), for further manipulation and loading into the model.


In [11]:
import pandas as pd


#Read in CSV
def GetRecording(path):
    recording_path = "../recordings/"
    file_name = input("Input Recording File Name")
    try:
        dataframe = pd.read_csv(recording_path + file_name + ".csv")
        return dataframe, file_name
    except: 
        print("Error Reading File: Check Spelling and Try Again")
        return 0
    
    
#Seperate each tracker to seperate dataframe

def GetColByName(dataframe):
    HMD = dataframe.loc[:, ["HMD0_tx", "HMD0_ty", "HMD0_tz"]]
    
    controller_1 = dataframe.loc[:, ['controller3_tx', 'controller3_ty', 'controller3_tz']]

    controller_2 = dataframe.loc[:, ['controller4_tx', 'controller4_ty', 'controller4_tz']]

    tracker_1 = dataframe.loc[:, ['generic7_tx', 'generic7_ty', 'generic7_tz']]

    tracker_2 = dataframe.loc[:, ['generic8_tx', 'generic8_ty', 'generic8_tz']]

    tracker_3 = dataframe.loc[:, ['generic9_tx', 'generic9_ty', 'generic9_tz']]

    joined = pd.concat([HMD,controller_1, controller_2, tracker_1 ,tracker_2 ,tracker_3], axis=1)
    return joined

def AssignTracker(dataframe):
    display(dataframe.iloc[0:1,:])
    trackerNum = 7
    for x in range(3):
        trackerStr = str(trackerNum)
        tracker = input('assign generic' + trackerStr)
        dataframe.rename(columns={'generic' + trackerStr + '_tx': tracker + '_x', 'generic' + trackerStr + '_ty': tracker + "_y", 'generic' + trackerStr + '_tz': tracker + '_z'}, inplace=True)
        trackerNum += 1
        
    controllerNum = 3
    for x in range(2):
        controllerStr = str(controllerNum)
        controller = input('assign controller' + controllerStr)
        dataframe.rename(columns={'controller' + controllerStr + '_tx': controller + '_x', 'controller' + controllerStr + '_ty': controller + "_y", 'controller' + controllerStr + '_tz': controller + '_z'}, inplace=True)
        controllerNum += 1
    dataframe.rename(columns={'HMD0_tx': 'head_x', 'HMD0_ty': 'head_y', 'HMD0_tz': 'head_z'}, inplace=True)
    return dataframe

def GetDirectory():
    choice = input("train or test data:")
    if choice == "test":
        output_path = "../test_data/"
    else:
        output_path = "../train_data/"
    return output_path

def OrderFeatures(dataframe):
    head = dataframe.loc[:, ['head_x', 'head_y', 'head_z']]
    l_controller = dataframe.loc[:, ['l_controller_x', 'l_controller_y', 'l_controller_z']]
    r_controller = dataframe.loc[:, ['r_controller_x', 'r_controller_x', 'r_controller_x']]
    waist = dataframe.loc[:, ['waist_x', 'waist_y', 'waist_z']]
    r_foot = dataframe.loc[:, ['r_foot_x', 'r_foot_y', 'r_foot_z']]
    l_foot = dataframe.loc[:, ['l_foot_x', 'l_foot_y', 'l_foot_z']]
    reordered = pd.concat([head , r_controller, l_controller, waist, r_foot, l_foot], axis=1)
    return reordered

    
    
def WriteOutput(path, dataframe, filename):
    output_file = path + filename + "_trimmed.csv"
    dataframe.to_csv(output_file, index = False)
    print(file_name + " output to " + path)
    
    

In [17]:
dataframe, file_name = GetRecording(recording_path)
joined = GetColByName(dataframe)
renamed = AssignTracker(joined)
path = GetDirectory()
reordered = OrderFeatures(renamed)
WriteOutput(path, reordered, file_name)


Input Recording File Name sitting_standing_2


,HMD0_tx,HMD0_ty,HMD0_tz,controller3_tx,controller3_ty,controller3_tz,controller4_tx,controller4_ty,controller4_tz,generic7_tx,generic7_ty,generic7_tz,generic8_tx,generic8_ty,generic8_tz,generic9_tx,generic9_ty,generic9_tz
0,4.449445,161.019928,-7.38734,-22.271193,80.701996,-9.232992,23.949348,81.303978,-13.191509,27.482939,11.219478,-1.153433,1.71808,99.981514,-8.398783,-19.821882,10.897743,2.164054


assign generic7 r_foot
assign generic8 waist
assign generic9 l_foot
assign controller3 l_controller
assign controller4 r_controller
train or test data: test


sitting_standing_2 output to ../test_data/


# Data Normalization

## Data Scaling

The new CSV is loaded into memory, chosen through an input prompt
The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).
These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"


#read in formatted CSV
def ReadCSV():
    file_name = input("Input File Name")
    try:
        dataframe = pd.read_csv(output_path + file_name + ".csv")
        print("Dataframe created")
    except:
        print("Error Reading File")
    return dataframe

def SplitFeaturesLabels(dataframe):
    x = dataframe.iloc[:, 0:9]
    y = dataframe.iloc[:, 9:18]
    return x, y

#Load data into Numpy array
def LoadArray(x, y):
    x_array = np.array(x)
    y_array = np.array(y)
    return x_array, y_array


def NormalizeValues (x, y):
    x =  np.divide(x, 100)
    y =  np.divide(y, 100)
    return x, y

def SampleSize(x, y):
    x_samples = x[0:900,:]
    y_samples = y[0:900,:]
    return x_samples, y_samples

def RoundValues(x, y): 
    x_rounded = np.around(x, 3)
    y_rounded = np.around(y, 3)
    return x_rounded, y_rounded



    
    

    





# Create Training Data

In [3]:
    
#load train data from csv
train_dataframe = ReadCSV()

#split features and labels into seperate dataframes
x_train_df, y_train_df = SplitFeaturesLabels(train_dataframe)

#convert features and labels to numpy array
x_train, y_train = LoadArray(x_train_df, y_train_df)

#Divide values in array by 100
x_train_normalized, y_train_normalized = NormalizeValues(x_train, y_train)


x_samples, y_samples = SampleSize(x_train_normalized, y_train_normalized)

print(x_samples.shape, x_samples)
print(y_samples.shape, y_samples)
print(x_train_df)
#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)





Input File Name walking_1_train
Dataframe created
(900, 9) [[ 0.00396347  1.55321396 -0.09726781 ... -0.17044104  0.79216263
  -0.12728346]
 [ 0.00396347  1.55321396 -0.0974095  ... -0.1698889   0.79222771
  -0.12750142]
 [ 0.00419568  1.55321396 -0.09760312 ... -0.16916578  0.79246605
  -0.12798414]
 ...
 [-0.02651245  1.59055893 -0.13678305 ... -0.11887306  0.75262321
   0.13336996]
 [-0.03070768  1.59075287 -0.13581562 ... -0.13185926  0.75569618
   0.13421907]
 [-0.03509516  1.5909549  -0.13504914 ... -0.14492013  0.75884453
   0.13558317]]
(900, 9) [[ 0.03550047  0.99325623 -0.06203985 ... -0.17026985  0.09899879
   0.01698667]
 [ 0.03546809  0.99353798 -0.06230913 ... -0.17021811  0.09892341
   0.01698667]
 [ 0.0358079   0.99352867 -0.06240509 ... -0.17014109  0.09882551
   0.01698667]
 ...
 [-0.07211315  1.00282028 -0.1262004  ... -0.08610182  0.09165998
   0.05806892]
 [-0.07784638  1.00439491 -0.1233117  ... -0.08573814  0.09138713
   0.05814877]
 [-0.08183085  1.0055632  -0.1

# Reshape Training Data

In [7]:

def ReshapeData(x, y):
    x_reshaped = np.expand_dims(x, axis=1)
    y_reshaped = np.expand_dims(y, axis=1)

    return x_reshaped, y_reshaped


In [8]:
x_train, y_train = ReshapeData(x_samples, y_samples)

print(x_train.shape, y_train.shape)

print(x_train.shape[1])

print(x_train.shape[2])


(900, 1, 9) (900, 1, 9)
1
9


# Create Test / Validation Data

In [17]:
test_dataframe = ReadCSV()

#split features and labels into seperate dataframes
x_test_df, y_test_df = SplitFeaturesLabels(test_dataframe)

#convert features and labels to numpy array
x_test, y_test = LoadArray(x_test_df, y_test_df)

#Divide values in array by 100
x_test_normalized, y_test_normalized = NormalizeValues(x_test, y_test)

x_test_samples, y_test_samples = SampleSize(x_test_normalized, y_test_normalized)

x_test, y_test = ReshapeData(x_test_samples, y_test_samples)

print(x_test.shape, y_test.shape)

print(x_test.shape[1])

Input File Name jumping_1_trimmed
Dataframe created
(900, 1, 9) (900, 1, 9)
1


# Model Creation and Training

In [10]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import tensorflow as tf
from keras.layers import LSTM, GRU

from keras.layers.core import Dense, Activation, Dropout

tf.keras.backend.set_floatx('float64')


model = Sequential()
model.add(GRU(9, return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(9, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(9, activation = "tanh"))

model.compile(loss='mse', optimizer='adam')

print ('model compiled')

print (model.summary())

model compiled
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 9)              540       
                                                                 
 dropout (Dropout)           (None, 1, 9)              0         
                                                                 
 gru_1 (GRU)                 (None, 9)                 540       
                                                                 
 dropout_1 (Dropout)         (None, 9)                 0         
                                                                 
 dense (Dense)               (None, 9)                 90        
                                                                 
Total params: 1,170
Trainable params: 1,170
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
x = [x_train, x_test]
y = [y_train, y_test]
y_pntr = 0
for array in x:
    model.fit(array, y[y_pntr], epochs=100,batch_size=10)
    y_pntr=+1

Epoch 1/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0768
Epoch 2/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0769
Epoch 3/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0766
Epoch 4/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0770
Epoch 5/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0766
Epoch 6/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0769
Epoch 7/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0769
Epoch 8/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0765
Epoch 9/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0768
Epoch 10/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0768
Epoch 11/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0768
Epoch 12/100
90/90 [==============================] - 0s 1ms/step - loss: 0.0765
Epoch 13/100
90/90 [=================

In [22]:
model.evaluate(x_test, y_test, batch_size=32)

29/29 [==============================] - 0s 1ms/step - loss: 0.0262


0.026248278220494588

In [39]:
predictions = model.predict(x_test[0:1,])
print(y_test[0:1,:,:])



[[[ 0.06434262  0.98289474 -0.04098022  0.31927406  0.13133705
    0.07496488 -0.17612398  0.09263622  0.05688983]]]


In [40]:
print("predictions shape:", predictions.shape)

prediction_DF = pd.DataFrame(predictions, columns=["Waist_X", "Waist_Y", "Waist_Z", "Rigth_Foot_X", "Right_Foot_Y", "Right_Foot_Z", "Left_Foot_X", "Left_Foot_Y", "Left_Foot_Z"])

display(prediction_DF)


predictions shape: (1, 9)


,Waist_X,Waist_Y,Waist_Z,Rigth_Foot_X,Right_Foot_Y,Right_Foot_Z,Left_Foot_X,Left_Foot_Y,Left_Foot_Z
0,0.110638,0.975918,0.30065,0.140573,0.128621,0.317704,0.090617,0.1229,0.281884
